# Imports

In [4]:
import numpy as np
import pandas as pd

## Initialize functions

In [9]:
def categorize(df, categories):
    for category in categories:
        df[category] = pd.Categorical(df[category])
    return df

def numberize(df, numerics):
    for numeric in numerics:
        df[numeric] = df[numeric].astype("float")
    return df

def clean_year(year):
    return 2024 - year

def get_category_name(df, column, code):
    if column in df.columns and pd.api.types.is_categorical_dtype(df[column]):
        cat = df[column].cat
        categories = cat.categories
        if 0 <= code < len(categories):
            category_name = categories[code]
            return category_name
        else:
            return None
    else:
        raise ValueError(f"Column '{column}' is either missing or not of 'category' dtype")



## load and prepare the data

In [10]:
df = pd.read_csv("flatfox.csv", low_memory=False)
df.drop(df[df["offer_type"] != "RENT"].index, inplace=True)
# remove columns with wrong year
df.drop(df[df["year_built"] < 1800].index, inplace=True)
df.drop(df[df["year_built"] > 2030].index, inplace=True)
# NaN  in floors is replaced by 0
# year_built has unrealistic values (for example 19 Million)
#df['year_built'] = df['year_built'].apply(clean_year_built)

df.fillna({"year_built": df["year_built"].mean(), "floor": 0.0, "year_renovated": df["year_built"], "number_of_rooms": 0.0, "livingspace": 0.0}, inplace=True)

df.loc[df["year_renovated"] < 1800, "year_renovated"] = df["year_built"]

df["year_renovated"] = df["year_renovated"].apply(clean_year)
df["year_built"] = df["year_built"].apply(clean_year)


df.dropna(subset=["price_display"], inplace=True)
df.drop([
    "pk",
    "slug",
    "url",
    "short_url",
    "moving_date",
    "rent_net",
    "rent_charges",
    "rent_gross",
    "offer_type",
    "submit_url",
    "status",
    "created",
    "reference",
    "ref_property",
    "ref_house",
    "ref_object",
    "alternative_reference",
    "published",
    "short_title",
    "public_title",
    "pitch_title",
    "description_title",
    "description",
    "attributes",
    "public_address",
    "video_url",
    "tour_url",
    "website_url",
    "live_viewing_url",
    "cover_image",
    "images",
    "documents",
    "agency",
    "rent_title",
    "surface_property",
    "surface_living",
    "surface_usable",
    "surface_usable_minimum",
    "volume",
    "space_display",
    "street"], axis=1, inplace=True)

categories = [
    "object_category",
    "object_type",
    "price_display_type",
    "price_unit",
    "city",
    "moving_date_type"
]
numerics = [
    "is_furnished",
    "is_temporary",
    "is_selling_furniture",
    "reserved"
]
categorize(df, categories)
numberize(df, numerics)
df.describe()
#df.head()
#df.isna().sum()
#df.dtypes



,price_display,number_of_rooms,floor,is_furnished,is_temporary,is_selling_furniture,zipcode,latitude,longitude,year_built,year_renovated,reserved,livingspace
count,20713.000000,20713.000000,20713.000000,20713.000000,20713.000000,20713.000000,20713.000000,20713.000000,20713.000000,20713.000000,20713.000000,20713.000000,20713.000000
mean,1581.338387,1.927027,1.298846,0.143050,0.065273,0.023801,5563.757592,47.159021,8.024401,38.998547,31.567108,0.000338,81.059962
std,1534.722897,1.839060,2.109293,0.350133,0.247013,0.152434,2743.831775,0.404267,0.847758,21.681407,19.387146,0.018381,673.658569
min,1.000000,0.000000,-8.000000,0.000000,0.000000,0.000000,1000.000000,45.826182,5.991881,-2.000000,-3.000000,0.000000,0.000000
25%,398.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3097.000000,46.956562,7.467781,38.946387,15.000000,0.000000,0.000000
50%,1460.000000,1.500000,1.000000,0.000000,0.000000,0.000000,5430.000000,47.346442,8.207061,38.946387,38.946387,0.000000,58.000000
75%,2141.000000,3.500000,2.000000,0.000000,0.000000,0.000000,8152.000000,47.430642,8.587571,38.946387,38.946387,0.000000,94.000000
max,62539.000000,10.500000,31.000000,1.000000,1.000000,1.000000,9657.000000,47.793652,10.364311,224.000000,224.000000,1.000000,90000.000000


count all years: 21040

count 1500-2024: 20725

count 1800-2024: 20672


Write clean data to csv

In [39]:
df.to_csv("flatfox_clean.csv", index=False)

## Create profiling report

In [40]:
from ydata_profiling import ProfileReport
ProfileReport(pd.read_csv("flatfox.csv", low_memory=False).drop(["created", "published"], axis=1), title="Flatfox Profiling").to_file("flatfox_profiling.html")
ProfileReport(pd.read_csv("flatfox_clean.csv", low_memory=False), title="Flatfox Profiling").to_file("flatfox_clean_profiling.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/home/lukaswipf/zhaw/semester5/DAAI/schweizerland/project/daai-gg-flatfox/.venv/lib/python3.12/site-packages/ydata_profiling/model/correlations.py:66: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'could not convert string to float: 'agr'')
  warnings.warn(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
# Call the function
city_code = 8

city_name = get_category_name(df, "object_type", city_code)

# Print out the result
print(f"The city corresponding to '{city_code}' is '{city_name}'.")

The city corresponding to '8' is 'BACHELOR_FLAT'.


C:\Users\lirla\AppData\Local\Temp\ipykernel_14612\3188197535.py:15: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if column in df.columns and pd.api.types.is_categorical_dtype(df[column]):


## Clean data without parking description

In [21]:
df2 = pd.read_csv("flatfox_no_parking_clean.csv", low_memory=False)
df2.describe()

,object_category,object_type,price_display,price_display_type,price_unit,number_of_rooms,floor,is_furnished,is_temporary,is_selling_furniture,zipcode,city,latitude,longitude,year_built,year_renovated,moving_date_type,reserved,livingspace
count,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000
mean,0.028233,1.771836,2122.884766,0.999829,0.000171,3.262410,1.990020,0.164961,0.072501,0.027721,5372.672637,923.062692,47.147571,7.993014,39.471739,29.888156,1.016888,0.000597,73.392613
std,0.165644,3.899079,1087.901432,0.013060,0.013060,1.322995,2.159625,0.371161,0.259326,0.164179,2696.155419,576.256038,0.413635,0.841675,23.461238,20.711948,0.727863,0.024429,48.428886
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,-5.000000,0.000000,0.000000,0.000000,1000.000000,0.000000,45.826182,5.991881,-1.000000,-1.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1480.000000,1.000000,0.000000,2.500000,1.000000,0.000000,0.000000,0.000000,3072.000000,412.000000,46.950762,7.445634,39.479863,9.000000,0.000000,0.000000,47.000000
50%,0.000000,0.000000,1870.000000,1.000000,0.000000,3.500000,2.000000,0.000000,0.000000,0.000000,4833.000000,892.000000,47.318822,7.909561,39.479863,39.479863,1.000000,0.000000,74.000000
75%,0.000000,0.000000,2500.000000,1.000000,0.000000,4.500000,3.000000,0.000000,0.000000,0.000000,8052.000000,1422.000000,47.434064,8.573304,39.479863,39.479863,2.000000,0.000000,97.000000
max,1.000000,19.000000,15000.000000,1.000000,1.000000,10.500000,31.000000,1.000000,1.000000,1.000000,9657.000000,1812.000000,47.768032,10.364311,224.000000,224.000000,2.000000,1.000000,1275.000000
